In [1]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=7bfb5785770f4c4e346e9baa4308eb5c470505877302cc9fefbc34dc3a4316b2
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [2]:
!pip install --upgrade joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from sentence_transformers import SentenceTransformer, models, util
import pandas as pd
import os

def createEmbedder():
    #embedding_method = "sentence-transformers/bert-base-nli-mean-tokens"
    #embedding_method = "gsarti/biobert-nli"
    embedding_method = "gsarti/scibert-nli"
    
    #embedder = SentenceTransformer(embedding_method)

    word_embedding_model = models.Transformer(embedding_method, max_seq_length=512, do_lower_case=True)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())    
    return SentenceTransformer(modules=[word_embedding_model, pooling_model])


In [4]:
inputFile = 'Questions_Generations_Annotations.csv'
df = pd.read_csv(inputFile, encoding= 'unicode_escape', index_col=False, names=['ID', 'Sample','Question','Question_Type','Question_Type1','Correct_Grammar','Suggestions'])
df

,ID,Sample,Question,Question_Type,Question_Type1,Correct_Grammar,Suggestions
0,NaN,Sample,Question,Question Type,Question Type,Correct Grammar,Suggestions
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID,Question / Sentence Samples,1 - Question \n0 - Not a Question,What / When / Why / How / Where / Who / Which ...,Second Label,1 - Correct Grammar\n0 - Incorrect Grammar,Correct grammar question / sentence if it is i...
3,AAG,Do you have to remove tartar from dogÃ¢â¬â¢s...,1,YN,NaN,1,NaN
4,AAJ,I recommend you look for enzymatic toothpaste ...,0,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...
5286,ZZY,Where does we buyer really sad sympathy cards,1,Where,NaN,0,Where do we buy really sad sympathy cards?
5287,ZZR,What animal are at risk?,1,What,NaN,0,Which animals are at risk?
5288,ZZS,What are poisonous to a dogs,1,What,NaN,0,What is poisonous to dogs?
5289,ZZW,It cause high blood sugars in dogs,0,NaN,NaN,0,It causes high blood sugar in dogs.


In [5]:
df.drop(index=df.index[:3], axis=0, inplace=True)

In [6]:
df = df.reset_index()
df

,index,ID,Sample,Question,Question_Type,Question_Type1,Correct_Grammar,Suggestions
0,3,AAG,Do you have to remove tartar from dogÃ¢â¬â¢s...,1,YN,NaN,1,NaN
1,4,AAJ,I recommend you look for enzymatic toothpaste ...,0,NaN,NaN,1,NaN
2,5,AAM,Why do you brush your cat's teeth? What are so...,1,Why,What,1,NaN
3,6,AAS,What was the correct answer to what I heard?,1,What,NaN,1,NaN
4,7,AAU,What was a dog once loved with?,1,What,NaN,1,NaN
...,...,...,...,...,...,...,...,...
5283,5286,ZZY,Where does we buyer really sad sympathy cards,1,Where,NaN,0,Where do we buy really sad sympathy cards?
5284,5287,ZZR,What animal are at risk?,1,What,NaN,0,Which animals are at risk?
5285,5288,ZZS,What are poisonous to a dogs,1,What,NaN,0,What is poisonous to dogs?
5286,5289,ZZW,It cause high blood sugars in dogs,0,NaN,NaN,0,It causes high blood sugar in dogs.


In [7]:
df['Question'].value_counts()

1    4481
0     807
Name: Question, dtype: int64

In [8]:
training_data = df[['Sample', 'Question']]

In [9]:
training_data.head()

,Sample,Question
0,Do you have to remove tartar from dogÃ¢â¬â¢s...,1
1,I recommend you look for enzymatic toothpaste ...,0
2,Why do you brush your cat's teeth? What are so...,1
3,What was the correct answer to what I heard?,1
4,What was a dog once loved with?,1


In [10]:
#Generating embedding for each question
embedder = createEmbedder()
training_data['embeddings'] = training_data.Sample.apply(lambda x: embedder.encode(x))

# #Writing to file
# outputFile = inputFile.replace('.csv','_embed.csv')
# training_data.embeddings.to_csv(outputFile, index=False)

<ipython-input-10-7d8ebcc02043>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['embeddings'] = training_data.Sample.apply(lambda x: embedder.encode(x))


In [11]:
training_data.head()

,Sample,Question,embeddings
0,Do you have to remove tartar from dogÃ¢â¬â¢s...,1,"[-0.86347234, 1.2074543, 0.5455055, 0.7629091,..."
1,I recommend you look for enzymatic toothpaste ...,0,"[0.3137987, 1.3157419, -0.6723612, 1.1914874, ..."
2,Why do you brush your cat's teeth? What are so...,1,"[0.632565, 1.8608861, -0.63003683, 1.1320025, ..."
3,What was the correct answer to what I heard?,1,"[-1.4644817, 1.6326419, -0.0247233, 0.5879964,..."
4,What was a dog once loved with?,1,"[-0.9674658, 1.7015969, -0.13893342, 0.1851079..."


In [12]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_train_smote, y_train_smote = sm.fit_resample(list(training_data.embeddings), training_data.Question.astype('int'))

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_smote, y_train_smote, test_size=0.33, random_state=42)

In [14]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()
sgd.fit(list(X_train), y_train)

SGDClassifier()

In [15]:
y_pred_sgd = sgd.predict(list(X_test))

In [16]:
from sklearn.linear_model import LogisticRegression


lr = LogisticRegression(max_iter = 1000)
lr.fit(list(X_train), y_train)

LogisticRegression(max_iter=1000)

In [17]:
y_pred_lr = lr.predict(list(X_test))

In [18]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(list(X_train), y_train)

RandomForestClassifier()

In [19]:
y_pred_rf = rf.predict(list(X_test))

In [20]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

nb_gaussian = GaussianNB()
nb_gaussian.fit(list(X_train), y_train)

nb_bern = BernoulliNB()
nb_bern.fit(list(X_train), y_train)

BernoulliNB()

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

pipe = Pipeline([
    ["scale",MinMaxScaler()],
    ["clf",MultinomialNB()]
])

pipe.fit(list(X_train), y_train)

Pipeline(steps=[('scale', MinMaxScaler()), ['clf', MultinomialNB()]])

In [22]:
y_pred_nbg = nb_gaussian.predict(list(X_test))

In [23]:
y_pred_nbb = nb_bern.predict(list(X_test))

In [24]:
y_pred_pipe = pipe.predict(list(X_test))

In [25]:
from sklearn.metrics import confusion_matrix, f1_score

In [26]:
confusion_matrix(y_true=y_test, y_pred=y_pred_sgd)

array([[1499,    2],
       [   8, 1449]])

In [27]:
confusion_matrix(y_true=y_test, y_pred=y_pred_lr)

array([[1501,    0],
       [   7, 1450]])

In [28]:
confusion_matrix(y_true=y_test, y_pred=y_pred_rf)

array([[1492,    9],
       [  17, 1440]])

In [29]:
confusion_matrix(y_true=y_test, y_pred=y_pred_nbg)

array([[1371,  130],
       [ 134, 1323]])

In [30]:
confusion_matrix(y_true=y_test, y_pred=y_pred_nbb)

array([[1324,  177],
       [ 154, 1303]])

In [31]:
confusion_matrix(y_true=y_test, y_pred=y_pred_pipe)

array([[1345,  156],
       [ 154, 1303]])

In [32]:
f1_score(y_true=y_test, y_pred=y_pred_sgd)

0.9965612104539203

In [33]:
f1_score(y_true=y_test, y_pred=y_pred_lr)

0.9975920192638459

In [34]:
f1_score(y_true=y_test, y_pred=y_pred_rf)

0.9910529938059188

In [35]:
f1_score(y_true=y_test, y_pred=y_pred_nbg)

0.909278350515464

In [36]:
f1_score(y_true=y_test, y_pred=y_pred_nbb)

0.8872999659516513

In [37]:
f1_score(y_true=y_test, y_pred=y_pred_pipe)

0.8936899862825789

In [38]:
sent = "are you sure"

sent_emb = embedder.encode(sent)

In [39]:
wr_sent = "What is the accelerated form of udysiufa decomposition that uses a sterile water solution?"

wr_emb = embedder.encode(wr_sent)

sgd.predict([wr_emb, sent_emb])

array([1, 1])

In [40]:
lr.predict([wr_emb, sent_emb])

array([1, 1])

In [41]:
rf.predict([wr_emb, sent_emb])

array([1, 1])

In [42]:
nb_gaussian.predict([wr_emb, sent_emb])

array([0, 1])

In [43]:
nb_bern.predict([wr_emb, sent_emb])

array([0, 1])

In [44]:
pipe.predict([wr_emb, sent_emb])

array([0, 1])

In [45]:
import joblib

joblib.dump(rf, "./random_forest.joblib")

['./random_forest.joblib']

In [46]:
rf_loaded = joblib.load('random_forest.joblib')

In [47]:
rf_loaded.predict([wr_emb])

array([1])